In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

import csv

In [2]:
client_id = "43c1aefc29484fab8b76593469ab4c44"
client_secret = "df001dead9514c7e8f65ab4dbfaee2ba"

In [3]:
# Autenticación
manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=manager)